In [50]:
from dsio.main import restream_dataframe
from dsio.anomaly_detectors import Percentile1D
import pandas as pd
import numpy as np
import scipy as sp
import plotly
import plotly.figure_factory as ff
import plotly.graph_objs as go
import scipy.signal as ss
from statistics import median, mean
from plotly import tools
from heapq import heappush, heappop

In [14]:
from __future__ import division
from pylab import plot, ylim, xlim, show, xlabel, ylabel, grid
from numpy import linspace, loadtxt, ones, convolve
import numpy as numpy

In [15]:
from selfpkg import readDB
from dataFilter import butter_lowpass_filter, butter_highpass_filter

In [2]:
from selfpkg import MinToMS
from directionCalculation import getDirection, getDirectionByRotation, getTurningPoint

In [34]:
timeConversionVal = 60000

In [18]:
def freqCal(df, peaksIndex):
    dfTime = df.iloc[peaksIndex] #get sub dataframe 
    prev = dfTime[0: 1] #cache the first 
    for i, _ in dfTime.iteritems():
        cur = dfTime[i] #cache cur
        dfTime[i] = abs(dfTime[i] - prev) #get difference
        prev = cur #set prev from cur (cur val we've cached before)
    return dfTime

In [19]:
def movingaverage(interval, window_size):
    window= numpy.ones(int(window_size))/float(window_size)
    return numpy.convolve(interval, window, 'same')

# start = MinToMS(15, df1)
# end = MinToMS(175, df1)

# startIndex = df.index[df['time'] == start].tolist()[0]
# endIndex = df.index[df['time'] == end].tolist()[0]

# x1 = df1.time[startIndex: endIndex]
# y1 = df1.flapper0[startIndex: endIndex]

# y_av1 = movingaverage(y1, 100)

In [150]:
trace0 = go.Scattergl(
    x=x1/timeConversionVal,
    y=y1,
    mode = 'markers',
    marker = dict(
        color = ('rgb(127, 127, 127)'),
        size = 3)
)

trace1 = go.Scattergl(
    x=x1/timeConversionVal,
    y=y_av1,
    line = dict(
        color = ('rgb(255, 0, 0)'))
)

data = [trace0, trace1]

layout = dict(title = 'flapper sensor',
                   xaxis = dict(title = 'time'),
                   yaxis = dict(title = 'flapper sensor output'))

fig = dict(data = data, layout = layout)
plotly.offline.plot(fig, filename='test4anomalyDetection.html')

'file:///Users/meng/Documents/dataPreprocessing/venv/source/test4anomalyDetection.html'

In [20]:
#function for generate flapper Chart
def flapperChartHighpass(df, start=None, end=None, filename='flapperChartLowHighpass'):
    
    if start is None:
        start = df.iloc[0]['time'].item()
    else:
        start = MinToMS(start, df)
    if end is None:
        end = df.tail(1)['time'].item()
    else:
        end = MinToMS(end, df)
    
    startIndex = df.index[df['time'] == start].tolist()[0] #get the index of start & end
    endIndex = df.index[df['time'] == end].tolist()[0]
    
    processedSignal0 = butter_highpass_filter(df.flapper0[startIndex-15000: endIndex], cutoff, fs, order)
    processedSignal1 = butter_highpass_filter(df.flapper1[startIndex-15000: endIndex], cutoff, fs, order)
    processedSignal2 = butter_highpass_filter(df.flapper2[startIndex-15000: endIndex], cutoff, fs, order)
    processedSignal3 = butter_highpass_filter(df.flapper3[startIndex-15000: endIndex], cutoff, fs, order)
    processedSignal4 = butter_highpass_filter(df.flapper4[startIndex-15000: endIndex], cutoff, fs, order)
    processedSignal5 = butter_highpass_filter(df.flapper5[startIndex-15000: endIndex], cutoff, fs, order)
    processedSignal6 = butter_highpass_filter(df.flapper6[startIndex-15000: endIndex], cutoff, fs, order)
    processedSignal7 = butter_highpass_filter(df.flapper7[startIndex-15000: endIndex], cutoff, fs, order)
    
    trace0 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal, 
        y=processedSignal0[15000:] + 8,
        name = 'flapper0',
        line = dict(
            color = ('rgb(255, 0, 0)'),
            width = 1)
    )
    trace1 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=processedSignal1[15000:]  + 6,
        name = 'flapper1',
        line = dict(
            color = ('rgb(0, 255, 0)'),
            width = 1)
    )
    trace2 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=processedSignal2[15000:]  + 4,
        name = 'flapper2',
        line = dict(
            color = ('rgb(0, 0, 255)'),
            width = 1)
    )
    trace3 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=processedSignal3[15000:]  + 2,
        name = 'flapper3',
        line = dict(
            color = ('rgb(255, 127, 127)'),
            width = 1)
    )
    trace4 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=processedSignal4[15000:]  - 2,
        name = 'flapper4',
        line = dict(
            color = ('rgb(255, 0, 255)'),
            width = 1)
    )
    trace5 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=processedSignal5[15000:]  - 4,
        name = 'flapper5',
        line = dict(
            color = ('rgb(255, 127, 0)'),
            width = 1)
    )
    trace6 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=processedSignal6[15000:]  - 6,
        name = 'flapper6',
        line = dict(
            color = ('rgb(127, 127, 127)'),
            width = 1)
    )
    trace7 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=processedSignal7[15000:]  - 8,
        name = 'flapper7',
        line = dict(
            color = ('rgb(0, 0, 0)'),
            width = 1)
    )
    
    data = [trace0, trace1, trace2, trace3, trace4, trace5, trace6, trace7]

    layout = dict(title = filename,
                       xaxis = dict(title = 'time(minutes)'),
                       yaxis = dict(title = 'friction')
                      )

    fig = dict(data = data, layout = layout)
    plotly.offline.plot(fig, filename = filename + '.html')

In [21]:
#function for generate flapper Chart
def flapperChartLowpass(df, start=None, end=None, filename='flapperChartLowpass'):
    
    if start is None:
        start = df.iloc[0]['time'].item()
    else:
        start = MinToMS(start, df)
    if end is None:
        end = df.tail(1)['time'].item()
    else:
        end = MinToMS(end, df)
    
    startIndex = df.index[df['time'] == start].tolist()[0] #get the index of start & end
    endIndex = df.index[df['time'] == end].tolist()[0]
    
    trace0 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal, 
        y=butter_lowpass_filter(df.flapper0[startIndex: endIndex], cutoff, fs, order) + 40,
        name = 'flapper0',
        line = dict(
            color = ('rgb(255, 0, 0)'),
            width = 1)
    )
    trace1 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=butter_lowpass_filter(df.flapper1[startIndex: endIndex], cutoff, fs, order) + 30,
        name = 'flapper1',
        line = dict(
            color = ('rgb(0, 255, 0)'),
            width = 1)
    )
    trace2 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=butter_lowpass_filter(df.flapper2[startIndex: endIndex], cutoff, fs, order) + 20,
        name = 'flapper2',
        line = dict(
            color = ('rgb(0, 0, 255)'),
            width = 1)
    )
    trace3 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=butter_lowpass_filter(df.flapper3[startIndex: endIndex], cutoff, fs, order) + 30,
        name = 'flapper3',
        line = dict(
            color = ('rgb(255, 127, 127)'),
            width = 1)
    )
    trace4 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=butter_lowpass_filter(df.flapper4[startIndex: endIndex], cutoff, fs, order) - 10,
        name = 'flapper4',
        line = dict(
            color = ('rgb(255, 0, 255)'),
            width = 1)
    )
    trace5 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=butter_lowpass_filter(df.flapper5[startIndex: endIndex], cutoff, fs, order) - 20,
        name = 'flapper5',
        line = dict(
            color = ('rgb(255, 127, 0)'),
            width = 1)
    )
    trace6 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=butter_lowpass_filter(df.flapper6[startIndex: endIndex], cutoff, fs, order) - 30,
        name = 'flapper6',
        line = dict(
            color = ('rgb(127, 127, 127)'),
            width = 1)
    )
    trace7 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=butter_lowpass_filter(df.flapper7[startIndex: endIndex], cutoff, fs, order) - 40,
        name = 'flapper7',
        line = dict(
            color = ('rgb(0, 0, 0)'),
            width = 1)
    )
    
    data = [trace0, trace1, trace2, trace3, trace4, trace5, trace6, trace7]#, trace8]

    layout = dict(title = filename,
                       xaxis = dict(title = 'time(minutes)'),
                       yaxis = dict(title = 'friction')
                      )

    fig = dict(data = data, layout = layout)
    plotly.offline.plot(fig, filename = filename + '.html')

In [474]:
start = 77
end = 85

In [466]:
filename1 = 'flapperLowpassSH'
flapperChartLowpass(df, start, end, filename=filename1)

In [477]:
filename2 = 'flapperHighpassSH'
flapperChartHighpass(df, start, end, filename=filename2)

In [22]:
#function for find peaks
def findPeaks(df, start, end, dfFlapper):
    startIndex = df.index[df['time'] == start].tolist()[0] #calculate index
    endIndex = df.index[df['time'] == end].tolist()[0]
    #return a list of peaks index from scipy.signal.find_peaks
    peaksPosIndex, _ = ss.find_peaks(dfFlapper[startIndex: endIndex].values)#, height = 20, threshold = None, distance = 24)  
    peaksNegIndex, _ = ss.find_peaks(-dfFlapper[startIndex: endIndex].values)#, height = 20, threshold = None, distance = 24)
    return peaksPosIndex, peaksNegIndex   

In [23]:
def freqCal(df, peaksIndex):
    dfTime = df.iloc[peaksIndex] #get sub dataframe 
    prev = dfTime[0: 1] #cache the first 
    for i, _ in dfTime.iteritems():
        cur = dfTime[i] #cache cur
        dfTime[i] = abs(dfTime[i] - prev) #get difference
        prev = cur #set prev from cur (cur val we've cached before)
    return dfTime

In [72]:
cutoff = 1
fs = 50
order = 5
timeConversionVal = 60000

In [24]:
#function for generating acc chart with peaks
def flapperAnomalyDetection(df, dfFlapper, start=None, end=None, filename='flapperAnomalyDetection'):
    
    threshold = 0.96
    errorPercentage = 0.1
    
    if start is None:
        start = df.iloc[0]['time'].item()
    else:
        start = MinToMS(start, df)
    if end is None:
        end = df.tail(1)['time'].item()
    else:
        end = MinToMS(end, df)
        
    #calculate index from time
    startIndex = df.index[df['time'] == start].tolist()[0]
    endIndex = df.index[df['time'] == end].tolist()[0]
    
    #low pass filter
    processedSignal = butter_highpass_filter(dfFlapper[startIndex-15000: endIndex], cutoff, fs, order)
    processedSignal = processedSignal[15000:]
    #peaks index
    peaksPosIndex, _ = ss.find_peaks(processedSignal, threshold = 2, distance = 50)#, height = 7, distance = 20)
    peaksNegIndex, _ = ss.find_peaks(-processedSignal, threshold = 2, distance = 5000)#, height = 7, distance = 20) 
    
    peaksPosIndexPlus = []
    peaksNegIndexPlus = []
    # index for Time (
    #time index is seperated from peak index. Because peak index from 0 after we do low pass filtering. But time depends on your input
    for i in range(len(peaksPosIndex)):
        peaksPosIndexPlus.append(peaksPosIndex[i] + startIndex)
    for i in range(len(peaksNegIndex)):
        peaksNegIndexPlus.append(peaksNegIndex[i] + startIndex)
########################################signal & peaks#########################################
    trace0 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=processedSignal,
        name = 'acc',
        line = dict(
            color = ('rgb(0, 0, 255)'),
            width = 1)
    )
             
    trace2 = go.Scattergl(
        x=df.time[peaksPosIndexPlus]/timeConversionVal,
        y=processedSignal[peaksPosIndex],
        mode = 'markers',
        name = 'peaks',
        marker = dict(
            color = ('rgb(127, 127, 127)'))
    )
    trace3 = go.Scattergl(
        x=df.time[peaksNegIndexPlus]/timeConversionVal,
        y=processedSignal[peaksNegIndex],
        mode = 'markers',
        name = 'peaks',
        marker = dict(
            color = ('rgb(127, 127, 127)'))
    )
########################################signal & peaks#########################################

####################################magnitude abnormal detection################################   
    #get median of peaks magnitude

    sortedProcessedSignalPos = sorted(processedSignal[peaksPosIndex])
    sortedProcessedSignalNeg = sorted(processedSignal[peaksNegIndex])
    medianPos = median(sortedProcessedSignalPos)
    medianNeg = median(sortedProcessedSignalNeg)
    thresholdPos = sortedProcessedSignalPos[(int)(threshold * len(sortedProcessedSignalPos))]
    thresholdNeg = sortedProcessedSignalNeg[(int)((1-threshold) * len(sortedProcessedSignalNeg))]
    
    #calcute index of time   
    posIndexTime = list(i for i in peaksPosIndexPlus if processedSignal[i - startIndex] > thresholdPos and abs(processedSignal[i - startIndex] - medianPos) > abs(errorPercentage * medianPos))
    negIndexTime = list(i for i in peaksNegIndexPlus if processedSignal[i - startIndex] < thresholdNeg and abs(processedSignal[i - startIndex] - medianNeg) > abs(errorPercentage * medianNeg))
    
    #caculate the index of peaks
    posIndexSignal = [i-startIndex for i in posIndexTime]
    negIndexSignal = [i-startIndex for i in negIndexTime]
    
    trace4 = go.Scattergl(
        x=df.time[posIndexTime] / timeConversionVal,
        y=processedSignal[posIndexSignal],
        mode = 'markers',
        name = 'peaks',
        marker = dict(
            color = ('rgb(255, 0, 0)'))
    )
    
    trace5 = go.Scattergl(
        x=df.time[negIndexTime] / timeConversionVal,
        y=processedSignal[negIndexSignal],
        mode = 'markers',
        name = 'peaks',
        marker = dict(
            color = ('rgb(255, 0, 0)'))
    )
####################################magnitude abnormal detection################################  
    
####################################time interval abnormal detection##############################
    dfPosTime = freqCal(df.time, peaksPosIndexPlus) #get time interval list
    dfNegTime = freqCal(df.time, peaksNegIndexPlus)
    
    sortedTimePos = sorted(dfPosTime)
    sortedTimeNeg = sorted(dfNegTime)
    
    medianPosInterval = median(sorted(dfPosTime)) #calculate the median of time interval
    medianNegInterval = median(sorted(dfNegTime))
    
    thresholdPosInterval = sortedTimePos[(int)((1-threshold) * len(sortedTimePos))] #calculate the median of time interval
    thresholdNegInterval = sortedTimeNeg[(int)((1-threshold) * len(sortedTimeNeg))]
        
    posIndexTimeInterval = []
    negIndexTimeInterval = []
    #calculate the index of time
    for i in range(len(peaksPosIndexPlus)):
        diff = abs(df.time[peaksPosIndexPlus[i]] - df.time[peaksPosIndexPlus[i-1]])
        if  diff < thresholdPosInterval and diff < (1 - errorPercentage) * medianPosInterval:
            posIndexTimeInterval.append(peaksPosIndexPlus[i])
    for i in range(len(peaksNegIndexPlus)):
        diff = abs(df.time[peaksNegIndexPlus[i]] - df.time[peaksNegIndexPlus[i-1]])
        if  diff < thresholdNegInterval and diff < (1 - errorPercentage) * medianNegInterval:
            negIndexTimeInterval.append(peaksNegIndexPlus[i])
    #caculate the index of peaks
    negIndexSignalInterval = [i-startIndex for i in negIndexTimeInterval]
    posIndexSignalInterval = [i-startIndex for i in posIndexTimeInterval]
    
    trace6 = go.Scattergl(
        x=df.time[posIndexTimeInterval] / timeConversionVal,
        y=processedSignal[posIndexSignalInterval],
        mode = 'markers',
        name = 'peaks',
        marker = dict(
            color = ('rgb(0, 255, 0)'))
    )
    
    trace7 = go.Scattergl(
        x=df.time[negIndexTimeInterval] / timeConversionVal,
        y=processedSignal[negIndexSignalInterval],
        mode = 'markers',
        name = 'peaks',
        marker = dict(
            color = ('rgb(0, 255, 0)'))
    )
####################################time interval abnormal detection##############################
    
    data = [trace0, trace2, trace3, trace4, trace5, trace6, trace7]
    layout = dict(title = filename,
                       xaxis = dict(title = 'time(MS)'),
                       yaxis = dict(title = 'flapper sensor output'))

    fig = dict(data = data, layout = layout)
    plotly.offline.plot(fig, filename=filename + '.html')

In [71]:
dfFlapper = df['flapper0']
start = 77
end = 85
flapperAnomalyDetection(df, dfFlapper, start, end, 'flapper0AnomalyDetection')

In [25]:
#function for generate flapper Chart
def flapperProcessedChart(df, start=None, end=None, filename='flapperChart'):
    
    if start is None:
        start = df.iloc[0]['time'].item()
    else:
        start = MinToMS(start, df)
    if end is None:
        end = df.tail(1)['time'].item()
    else:
        end = MinToMS(end, df)
    
    startIndex = df.index[df['time'] == start].tolist()[0] #get the index of start & end
    endIndex = df.index[df['time'] == end].tolist()[0]
    
        #low pass filter
    processedSignal = butter_lowpass_filter(dfFlapper[startIndex-15000: endIndex], 5, 50, 5)
    processedSignal = processedSignal[15000:]
    #peaks index
    peaksPosIndex, _ = ss.find_peaks(processedSignal, prominence = 1)
    peaksNegIndex, _ = ss.find_peaks(-processedSignal, prominence = 1)
    
    peaksPosIndexPlus = []
    peaksNegIndexPlus = []
    # index for Time (
    #time index is seperated from peak index. Because peak index from 0 after we do low pass filtering. But time depends on your input
    for i in range(len(peaksPosIndex)):
        peaksPosIndexPlus.append(peaksPosIndex[i] + startIndex)
    for i in range(len(peaksNegIndex)):
        peaksNegIndexPlus.append(peaksNegIndex[i] + startIndex)
########################################signal & peaks#########################################


    y=processedSignal[peaksPosIndex]
    
    mask = [-1, 1] 
    y = []
    for i in [0,1,2,3,4,5,6,7]:
        tmp = butter_lowpass_filter(np.convolve(df['flapper' + str(i)][startIndex: endIndex], mask), 5, 50, 5)
        y.append(tmp)
        
    trace0 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal, 
        y=y[0] + 40,
        name = 'flapper0',
        line = dict(
            color = ('rgb(255, 0, 0)'),
            width = 1)
    )
    trace1 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=y[1] + 30,
        name = 'flapper1',
        line = dict(
            color = ('rgb(0, 255, 0)'),
            width = 1)
    )
    trace2 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=y[2] + 20,
        name = 'flapper2',
        line = dict(
            color = ('rgb(0, 0, 255)'),
            width = 1)
    )
    trace3 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=y[3]+ 10,
        name = 'flapper3',
        line = dict(
            color = ('rgb(255, 255, 255)'),
            width = 1)
    )
    trace4 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=y[4] - 10,
        name = 'flapper4',
        line = dict(
            color = ('rgb(255, 0, 255)'),
            width = 1)
    )
    trace5 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=y[5],
        name = 'flapper5',
        line = dict(
            color = ('rgb(255, 127, 0)'),
            width = 1)
    )
    trace6 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=y[6] - 30,
        name = 'flapper6',
        line = dict(
            color = ('rgb(127, 127, 127)'),
            width = 1)
    )
    trace7 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=y[7] - 40,
        name = 'flapper7',
        line = dict(
            color = ('rgb(0, 0, 0)'),
            width = 1)
    )
    
    trace8 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=y[7] - 50,
        name = 'flapper7',
        line = dict(
            color = ('rgb(0, 0, 0)'),
            width = 1)
    )

    #data = [trace0, trace1, trace2, trace3, trace4, trace5, trace6, trace7, trace8]
    data = [trace5]

    layout = dict(title = filename,
                       xaxis = dict(title = 'Time(minutes)', range = [52.01, 52.8]),
                       yaxis = dict(title = 'Friction', range=[-3, 3])
                      )

    fig = dict(data = data, layout = layout)
    plotly.offline.plot(fig, filename = filename + '.html') #offline generate

In [26]:
#function for generating acc chart with peaks
def flapperAnomalyDetectionSingle(df, dfFlapper, start=None, end=None):
    
    threshold = 0.96
    errorPercentage = 0.1
    
    if start is None:
        start = df.iloc[0]['time'].item()
    else:
        start = MinToMS(start, df)
    if end is None:
        end = df.tail(1)['time'].item()
    else:
        end = MinToMS(end, df)
        
    #calculate index from time
    startIndex = df.index[df['time'] == start].tolist()[0]
    endIndex = df.index[df['time'] == end].tolist()[0]
    
    #low pass filter
    processedSignal = butter_lowpass_filter(dfFlapper[startIndex-15000: endIndex], 10, 50, 5)
    processedSignal = processedSignal[15000:]
    #peaks index
    peaksPosIndex, _ = ss.find_peaks(processedSignal, prominence = 1)
    peaksNegIndex, _ = ss.find_peaks(-processedSignal, prominence = 1)
    
    peaksPosIndexPlus = []
    peaksNegIndexPlus = []
    # index for Time (
    #time index is seperated from peak index. Because peak index from 0 after we do low pass filtering. But time depends on your input
    for i in range(len(peaksPosIndex)):
        peaksPosIndexPlus.append(peaksPosIndex[i] + startIndex)
    for i in range(len(peaksNegIndex)):
        peaksNegIndexPlus.append(peaksNegIndex[i] + startIndex)
########################################signal & peaks#########################################
    trace0 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=processedSignal,
        name = 'acc',
        line = dict(
            color = ('rgb(0, 0, 255)'),
            width = 1)
    )
             
    trace2 = go.Scattergl(
        x=df.time[peaksPosIndexPlus]/timeConversionVal,
        y=processedSignal[peaksPosIndex],
        mode = 'markers',
        name = 'peaks',
        marker = dict(
            color = ('rgb(127, 127, 127)'))
    )
    trace3 = go.Scattergl(
        x=df.time[peaksNegIndexPlus]/timeConversionVal,
        y=processedSignal[peaksNegIndex],
        mode = 'markers',
        name = 'peaks',
        marker = dict(
            color = ('rgb(127, 127, 127)'))
    )
# ########################################signal & peaks#########################################

# ####################################magnitude abnormal detection################################   
#     #get median of peaks magnitude

#     sortedProcessedSignalPos = sorted(processedSignal[peaksPosIndex])
#     sortedProcessedSignalNeg = sorted(processedSignal[peaksNegIndex])
    
#     if len(sortedProcessedSignalPos) == 0:
#         medianPos = 0
#     else:
#         medianPos = median(sortedProcessedSignalPos)
#     if len(sortedProcessedSignalNeg) == 0:
#         medianNeg = 0
#     else:
#         medianNeg = median(sortedProcessedSignalNeg)
        
#     thresholdPos = sortedProcessedSignalPos[(int)(threshold * (len(sortedProcessedSignalPos)-1))]
#     thresholdNeg = sortedProcessedSignalNeg[(int)((1-threshold) * (len(sortedProcessedSignalNeg)-1))]
    
#     #calcute index of time   
#     posIndexTime = list(i for i in peaksPosIndexPlus if processedSignal[i - startIndex] > thresholdPos and abs(processedSignal[i - startIndex] - medianPos) > abs(errorPercentage * medianPos))
#     negIndexTime = list(i for i in peaksNegIndexPlus if processedSignal[i - startIndex] < thresholdNeg and abs(processedSignal[i - startIndex] - medianNeg) > abs(errorPercentage * medianNeg))
    
#     #caculate the index of peaks
#     posIndexSignal = [i-startIndex for i in posIndexTime]
#     negIndexSignal = [i-startIndex for i in negIndexTime]
    
#     trace4 = go.Scattergl(
#         x=df.time[posIndexTime] / timeConversionVal,
#         y=processedSignal[posIndexSignal],
#         mode = 'markers',
#         name = 'peaks',
#         marker = dict(
#             color = ('rgb(255, 0, 0)'))
#     )
    
#     trace5 = go.Scattergl(
#         x=df.time[negIndexTime] / timeConversionVal,
#         y=processedSignal[negIndexSignal],
#         mode = 'markers',
#         name = 'peaks',
#         marker = dict(
#             color = ('rgb(255, 0, 0)'))
#     )
# ####################################magnitude abnormal detection################################  
    
# ####################################time interval abnormal detection##############################
#     dfPosTime = freqCal(df.time, peaksPosIndexPlus) #get time interval list
#     dfNegTime = freqCal(df.time, peaksNegIndexPlus)
    
#     sortedTimePos = sorted(dfPosTime)
#     sortedTimeNeg = sorted(dfNegTime)
    
#     if len(dfPosTime) == 0:
#         medianPosInterval = 0
#     else:
#         medianPosInterval = median(sorted(dfPosTime)) #calculate the median of time interval
#     if len(dfNegTime) == 0:
#         medianNegInterval = 0
#     else:
#         medianNegInterval = median(sorted(dfNegTime))
    
#     thresholdPosInterval = sortedTimePos[(int)((1-threshold) * (len(sortedTimePos)-1))] #calculate the median of time interval
#     thresholdNegInterval = sortedTimeNeg[(int)((1-threshold) * (len(sortedTimeNeg)-1))]
        
#     posIndexTimeInterval = []
#     negIndexTimeInterval = []
#     #calculate the index of time
#     for i in range(len(peaksPosIndexPlus)):
#         diff = abs(df.time[peaksPosIndexPlus[i]] - df.time[peaksPosIndexPlus[i-1]])
#         if  diff < thresholdPosInterval and diff < (1 - errorPercentage) * medianPosInterval:
#             posIndexTimeInterval.append(peaksPosIndexPlus[i])
#     for i in range(len(peaksNegIndexPlus)):
#         diff = abs(df.time[peaksNegIndexPlus[i]] - df.time[peaksNegIndexPlus[i-1]])
#         if  diff < thresholdNegInterval and diff < (1 - errorPercentage) * medianNegInterval:
#             negIndexTimeInterval.append(peaksNegIndexPlus[i])
#     #caculate the index of peaks
#     negIndexSignalInterval = [i-startIndex for i in negIndexTimeInterval]
#     posIndexSignalInterval = [i-startIndex for i in posIndexTimeInterval]
    
#     trace6 = go.Scattergl(
#         x=df.time[posIndexTimeInterval] / timeConversionVal,
#         y=processedSignal[posIndexSignalInterval],
#         mode = 'markers',
#         name = 'peaks',
#         marker = dict(
#             color = ('rgb(0, 255, 0)'))
#     )
    
#     trace7 = go.Scattergl(
#         x=df.time[negIndexTimeInterval] / timeConversionVal,
#         y=processedSignal[negIndexSignalInterval],
#         mode = 'markers',
#         name = 'peaks',
#         marker = dict(
#             color = ('rgb(0, 255, 0)'))
#     )
####################################time interval abnormal detection##############################
    
    data = [trace0, trace2, trace3]#, trace4, trace5, trace6, trace7]
    return data

In [244]:
order = 5 #order of filter (1-)
fs = 50       # sample rate, Hz
cutoff = 2 # desired cutoff frequency of the filter, Hz

In [27]:
def flapperAnomalyDetectionFrontAllinOne(df, start=None, end=None, filename='flapperAnomalyDetectionFrontAllinOne'):
    fig = tools.make_subplots(rows=4, cols=1, subplot_titles=('flapper0', 'flapper1', 'flapper2', 'flapper3'))#, 'flapper4', 'flapper5', 'flapper6',  'flapper7'))
    for i in [0, 1, 2, 3]:
        dfFlapper = df['flapper' + str(i)]
        data = flapperAnomalyDetectionSingle(df, dfFlapper, start, end)
        for trace in data:
            fig.append_trace(trace, i+1, 1)
    
    plotly.offline.plot(fig, filename = filename + '.html')

In [28]:
def flapperAnomalyDetectionBackAllinOne(df, start=None, end=None, filename='flapperAnomalyDetectionBackAllinOne'):
    fig = tools.make_subplots(rows=4, cols=1, subplot_titles=('flapper4', 'flapper5', 'flapper6', 'flapper7'))#, 'flapper4', 'flapper5', 'flapper6',  'flapper7'))
    for i in [4, 5, 6, 7]:
        dfFlapper = df['flapper' + str(i)]
        data = flapperAnomalyDetectionSingle(df, dfFlapper, start, end)
        for trace in data:
            fig.append_trace(trace, i-3, 1)
    
    plotly.offline.plot(fig, filename = filename + '.html')

In [30]:
df = readDB("sqlite:///SouthHarrison.db")

In [35]:
flapperAnomalyDetectionBackAllinOne(df, 70, 80, filename='flapperAnomalyDetectionBackAllinOne')

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]



In [ ]:
flapperAnomalyDetectionFrontAllinOne(df, 70, 80, filename='flapperAnomalyDetectionFrontAllinOne')

In [36]:
from scipy.signal import butter, lfilter, freqz
def butter_bandstop_filter(data, lowcut, highcut, fs, order):

    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq

    i, u = butter(order, [low, high], btype='bandstop')
    y = lfilter(i, u, data)
    return y

In [37]:
#function for generate flapper Chart
def flapperChartBandBlock(df, start=None, end=None, filename='flapperChartBandBlock'):
    
    if start is None:
        start = df.iloc[0]['time'].item()
    else:
        start = MinToMS(start, df)
    if end is None:
        end = df.tail(1)['time'].item()
    else:
        end = MinToMS(end, df)
    
    startIndex = df.index[df['time'] == start].tolist()[0] #get the index of start & end
    endIndex = df.index[df['time'] == end].tolist()[0]
    
    #signalProcessed = butter_lowpass_filter(df.flapper2[startIndex: endIndex], 6, fs, order) + butter_highpass_filter(df.flapper2[startIndex: endIndex], 14, fs, order)
    signalProcessed = butter_bandstop_filter(df.flapper2[startIndex: endIndex], 6, 24, 50, 5)
    #signalProcessed = butter_highpass_filter(df.flapper2[startIndex: endIndex], 1, 50, 5)
    
    timeConversionVal = 60000
    trace0 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal, 
        y=signalProcessed,
        name = 'flapper2_bandStop',
        line = dict(
            color = ('rgb(255, 0, 0)'),
            width = 1)
    )
        
    trace1 = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=df.flapper2[startIndex: endIndex] - 10,
        name = 'flapper2',
        line = dict(
            color = ('rgb(0, 255, 0)'),
            width = 1)
    )


    data = [trace0, trace1]#, trace2, trace3, trace4, trace5, trace6, trace7]

    layout = dict(title = filename,
                       xaxis = dict(title = 'time(minutes)'),
                       yaxis = dict(title = 'friction',
                                        range = [0, 100]))

    fig = dict(data = data, layout = layout)
    plotly.offline.plot(fig, filename = filename + '.html') #offline generate

In [38]:
def FFT(df, dfFlapper, start=None, end=None, filename='test'):
    
    if start is None:
        start = df.iloc[0]['time'].item()
    else:
        start = MinToMS(start, df)
    if end is None:
        end = df.tail(1)['time'].item()
    else:
        end = MinToMS(end, df)
    
    startIndex = df.index[df['time'] == start].tolist()[0]
    endIndex = df.index[df['time'] == end].tolist()[0]
    
    fs = 50
    cutoff = 1
    order = 5

    freq = np.fft.fftfreq(len(df[startIndex: endIndex].index), d=0.02)
    sp = np.fft.fft(butter_highpass_filter(dfFlapper[startIndex: endIndex], cutoff, fs, order))
    #sp = np.fft.fft(dfFlapper[startIndex: endIndex])
#     print(freq)
#     print(sp.real)
#     sp = np.fft.fft(butter_bandstop_filter(dfFlapper[startIndex: endIndex], 6, 24, fs, order))


    trace0 = go.Scattergl(
        x=freq,
        y=sp.real,
        name = 'real',
        line = dict(
            color = ('rgb(0, 0, 255)'),
            width = 1)
    )
    
    data = [trace0]
    layout = dict(title = filename,
                       xaxis = dict(title = 'freq'),
                       yaxis = dict(title = 'magnitude'))
    fig = dict(data = data, layout = layout)
    fig['layout']['xaxis'].update(rangemode= 'nonnegative')
    plotly.offline.plot(fig, filename = filename + '.html')
    
# csv_database = 'sqlite:///SouthHarrison.db' 
# df = readDB(csv_database)

# dfFlapper = df.flapper7
# FFT(df, dfFlapper, 52.46, 52.48, 'test111')
# FFT(df, dfFlapper, 14.19, 14.21, 'test222')

In [106]:
from scipy.signal import savgol_filter
from statistics import stdev
def FFTsingle(df, dfFlapper, start=None, end=None):
    
    if start is None:
        start = df.iloc[0]['time'].item()
    else:
        start = MinToMS(start, df)
    if end is None:
        end = df.tail(1)['time'].item()
    else:
        end = MinToMS(end, df)
    
    startIndex = df.index[df['time'] == start].tolist()[0]
    endIndex = df.index[df['time'] == end].tolist()[0]
    
    fs = 50
    cutoff = 1
    order = 5

    freq = np.fft.fftfreq(len(df[startIndex: endIndex].index), d=0.02)
    sp = np.fft.fft(butter_highpass_filter(dfFlapper[startIndex: endIndex], cutoff, fs, order))
    #sp = np.fft.fft(dfFlapper[startIndex: endIndex] + np.sin(2 * 6.28 * df[startIndex: endIndex].index * 0.02 ))
#     sp = np.fft.fft(butter_bandstop_filter(dfFlapper[startIndex: endIndex], 1, 5, fs, order))

    trimFreq = []
    trimSp = []
    for i in range(len(freq)):
        if freq[i] >= 0 and freq[i] <= 100:
            trimFreq.append(freq[i])
            trimSp.append(sp[i].real)
            
    threshold = trimFreq[trimSp.index(max(trimSp))]
    trimFreq = []
    trimSp = []
    for i in range(len(freq)):
        if freq[i] > threshold:
            trimFreq.append(freq[i])
            trimSp.append(sp[i].real)
            
    m = max(trimSp)
    for i in range(len(trimSp)):
        trimSp[i] = trimSp[i] / m * 10
    
    trimSpFilter = savgol_filter(trimSp, 7, 3)
    
    list = []
    for i in range(len(trimFreq)):
        diff = abs(trimSp[i] - trimSpFilter[i])
        list.append(diff)
    
    sd = sum(list)
    
    trace0 = go.Scattergl(
        x=trimFreq,#freq,
        y=trimSp,#sp.real,
        name = 'real',
        line = dict(
            color = ('rgb(0, 0, 255)'),
            width = 1),
    )
    
    trace1 = go.Scattergl(
        x=trimFreq,#freq,
        y=trimSpFilter,
        name = 'fitting line',
        line = dict(
            color = ('rgb(0, 255, 0)'),
            width = 1),
    )
    
    data = [trace0, trace1]    
    return data, sd

In [117]:
def FFTFrontAllinOne(df, start=None, end=None, filename='FFTFrontAllinOne'):
    
    fig = tools.make_subplots(rows=4, cols=1, subplot_titles=('flapper0', 'flapper1', 'flapper2', 'flapper3'))
    list = []
    for i in [0, 1, 2, 3]:
        dfFlapper = df['flapper' + str(i)]
        data, sd = FFTsingle(df, dfFlapper, start, end)
        list.append(sd)
        for trace in data:
            fig.append_trace(trace, i+1, 1)
            fig['layout']['xaxis' + str(i+1)].update(title='Frequency(Hz)')
            fig['layout']['yaxis' + str(i+1)].update(title='Amplitude')
            
    fig['layout'].update(title='FFT for front flapper')
    plotly.offline.plot(fig, filename = filename + '.html')
    return list

In [118]:
def FFTBackAllinOne(df, start=None, end=None, filename='FFTBackAllinOne'):
    
    fig = tools.make_subplots(rows=4, cols=1, subplot_titles=('flapper4', 'flapper5', 'flapper6', 'flapper7'))
    list = []
    for i in [4, 5, 6, 7]:
        dfFlapper = df['flapper' + str(i)]
        data, sd = FFTsingle(df, dfFlapper, start, end)
        list.append(sd)
        for trace in data:
            fig.append_trace(trace, i-3, 1)
            fig['layout']['xaxis' + str(i-3)].update(title='Frequency(Hz)')
            fig['layout']['yaxis' + str(i-3)].update(title='Amplitude')
    
    fig['layout'].update(title='FFT for back flapper')
    plotly.offline.plot(fig, filename = filename + '.html')
    return list

In [47]:
#detect joint by both flapper & acc signal
def jointDetectionCompass(df, start=None, end=None, filename='jointDetection'):

    if start is None:
        start = df.iloc[0]['time'].item()
    else:
        start = MinToMS(start, df)
    if end is None:
        end = df.tail(1)['time'].item()
    else:
        end = MinToMS(end, df)
    
    startIndex = df.index[df['time'] == start].tolist()[0]
    endIndex = df.index[df['time'] == end].tolist()[0]
    
    #original flapper signal
    #add them all (get rid of bad flapper)
    z = []
    for i in range(8):
        if i == 0:
            z.append(df['flapper' + str(i)][startIndex: endIndex])
        else:
            z[0] += df['flapper' + str(i)][startIndex: endIndex]
    
    #lowpass & highpass for flapper signal
    y = []
    for i in range(8):
        if i == 0:
            y.append(butter_highpass_filter(butter_lowpass_filter(df['flapper' + str(i)][startIndex: endIndex], 5, 50, 5), 1, 50, 5))
        else:
            y[0] += butter_highpass_filter(butter_lowpass_filter(df['flapper' + str(i)][startIndex: endIndex], 5, 50, 5), 1, 50, 5)
    
    trace5 = go.Scattergl(
        x=df.time[startIndex: endIndex]/60000, 
        y=z[0] - 150,#res,
        name = 'flapper all in one',
        line = dict(
            color = ('rgb(0, 220, 0)'),
            width = 1)
    )
    
    x = df.compassX
    y = df.compassY
    z = df.compassZ

    v = -((x ** 2 + y ** 2 + z ** 2) ** (1/2))
    v = butter_highpass_filter(v, 1, 50, 5)
    v = abs(v)
    for i in range(len(v)):
        if abs(v[i]) < 4:
            v[i] = 0

    #generate peaks both for acceleration & time
    peaksAccIndex, _ = ss.find_peaks(v)
    peaksAccIndexPlus = [x + startIndex for x in peaksAccIndex]
    #same as flapper, we need to merge close peaks
    i = 0
    j = 1
    thresInterval = 50
    resAcc = []
    while(j < len(peaksAccIndexPlus)):
        if  j == len(peaksAccIndexPlus) - 1:
            resAcc.append(int((peaksAccIndexPlus[i] + peaksAccIndexPlus[j]) / 2))
            i = j
            j += 1       
        else:
            if peaksAccIndexPlus[j] - peaksAccIndexPlus[j-1] > thresInterval:
                j -= 1
                resAcc.append(int((peaksAccIndexPlus[i] + peaksAccIndexPlus[j]) / 2))
                j += 1
                i = j
                j += 1       
            else:
                j += 1
    
    traceCompassPoint = go.Scattergl(
        x=df.time[resAcc]/timeConversionVal,
        y=[-10] * len(resAcc),
        mode = 'markers',
        name = 'jointonCompass',
        marker = dict(
            color = ('rgb(127, 127, 127)'))
    )
    
    traceCompass = go.Scattergl(
        x=df.time[startIndex: endIndex]/timeConversionVal,
        y=v[startIndex:endIndex],
        name = 'compassSum',
        line = dict(
            color = ('rgb(127, 127, 127)'),
            width = 1)
    )
    
#     flapper0, flapper1, flapper2, flapper3, flapper4, flapper5, flapper6, flapper7 = flapperChart(df)
    
    data = [traceCompass, traceCompassPoint]#, traceTurning, traceSpecial, flapper0, flapper1, flapper2, flapper3, flapper4, flapper5, flapper6, flapper7]
    
    layout = dict(title = filename,
                       xaxis = dict(title = 'Time(minutes)'),
                       yaxis = dict(title = 'Friction')
                      )

    fig = dict(data = data, layout = layout)
#     plotly.offline.plot(fig, filename = filename + '.html')
    
    return resAcc #peaks index list

In [111]:
df = pd.read_csv('/Users/meng/Documents/dataPreprocessing/venv/source/sa_run01_trim.csv')

In [112]:
h = []
for i in range(10):
    heappush(h, (-1, 0))

In [113]:
peaks = jointDetectionCompass(df)
for i in peaks:
    mid = df.time[i] / timeConversionVal
    start = mid - 0.01
    end = mid + 0.01
    list1 = FFTFrontAllinOne(df, start, end)
    list2 = FFTBackAllinOne(df, start, end)
    
    m1 = max(list1)
    m2 = max(list2)
    m = max(m1, m2)
    
    tmp = heappop(h)
    if m >= tmp[0]:
        heappush(h, (m, mid))
    else:
        heappush(h, tmp)

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (

In [114]:
print(h)

[(24.549515127868577, 5.4811), (25.156763518419638, 6.0343), (28.479253308849284, 2.624233333333333), (26.11401560007404, 5.669233333333334), (54.06389053920704, 6.306416666666666), (35.685665895112095, 6.217083333333333), (31.24183331416299, 3.5410333333333335), (38.16262448465555, 4.093166666666667), (26.457079955810322, 3.456716666666667), (95.09716737886215, 6.863133333333334)]


In [119]:
# mid = 852217/60000
mid = 4.09
start = mid - 0.01
end = mid + 0.01
list1 = FFTFrontAllinOne(df, start, end)
list2 = FFTBackAllinOne(df, start, end)

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]



In [120]:
print(list1)
print(list2)

[1.0476882937056171, 32.023623443623556, 30.208033968445143, 16.76210626814099]
[6.658776625400019, 6.554863977383745, 1.7331131065376773, 14.805020615815405]
